In [1]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import time

from sklearn.model_selection import train_test_split


In [2]:
# load dataset
X_train=np.load("X_train.npy")
    
scaler=StandardScaler()
scaler.fit(X_train)
x_train=scaler.transform(X_train)
#===
Y_train=np.load("y_train.npy")

# -------------------------------------

X_test=np.load("X_test.npy")

#====z_score normalization
scaler2=StandardScaler()
scaler2.fit(X_test)
X_test=scaler2.transform(X_test)
#====
Y_test=np.load("y_test.npy")

print(X_train.shape, X_test.shape)

(5491, 16) (2354, 16)


In [3]:
'''#Concatenation 
all_data_X=np.concatenate((X_train, X_test),   axis=0)
all_data_Y=np.concatenate((Y_train, Y_test),   axis=0)

print(all_data_X.shape, all_data_Y.shape)'''

'#Concatenation \nall_data_X=np.concatenate((X_train, X_test),   axis=0)\nall_data_Y=np.concatenate((Y_train, Y_test),   axis=0)\n\nprint(all_data_X.shape, all_data_Y.shape)'

In [4]:
t1= time.time()
input_layer= Input(shape=(X_train.shape[1],))
encoded = Dense(units=512, activation='relu')(input_layer)
encoded = Dense(units=256, activation='relu')(encoded)
encoded = Dense(units=64, activation='relu')(encoded)
decoded = Dense(units=256, activation='relu')(encoded)
decoded = Dense(units=512, activation='relu')(decoded)
decoded = Dense(units=X_train.shape[1], activation='sigmoid')(decoded)
autoencoder=Model(input_layer, decoded)

autoencoder.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 16)                8208  

In [5]:
# get hidden layer
encoder = Model(input_layer, encoded)

encoder.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
Total params: 156,480
Trainable params: 156,480
Non-trainable params: 0
_________________________________________________________________


In [6]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
autoencoder.fit(X_train, X_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))


Epoch 1/20
22/22 [==============================] - 1s 17ms/step - loss: -322546499584.0000 - accuracy: 0.5760 - val_loss: -3.6104 - val_accuracy: 0.0191
Epoch 2/20
22/22 [==============================] - 0s 11ms/step - loss: -10301643685888.0000 - accuracy: 0.6039 - val_loss: -179.4829 - val_accuracy: 0.0191
Epoch 3/20
22/22 [==============================] - 0s 11ms/step - loss: -189872518201344.0000 - accuracy: 0.6039 - val_loss: -3282.1519 - val_accuracy: 0.0191
Epoch 4/20
22/22 [==============================] - 0s 11ms/step - loss: -1765353965223936.0000 - accuracy: 0.6039 - val_loss: -34924.3516 - val_accuracy: 0.0191
Epoch 5/20
22/22 [==============================] - 0s 11ms/step - loss: -8739209938468864.0000 - accuracy: 0.6039 - val_loss: -180269.9531 - val_accuracy: 0.0191
Epoch 6/20
22/22 [==============================] - 0s 11ms/step - loss: -30490093093388288.0000 - accuracy: 0.1637 - val_loss: -779793.0000 - val_accuracy: 0.2727
Epoch 7/20
22/22 [=====================

In [8]:
encoded_train = encoder.predict(X_train)
encoded_test = encoder.predict(X_test)
print(encoded_train.shape, encoded_test.shape)

(5491, 64) (2354, 64)


In [9]:
from sklearn.neural_network import BernoulliRBM
import numpy as np
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

'''digits = datasets.load_digits()
X = np.asarray(digits.data, 'float32')
Y = digits.target
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=0)'''

logistic = linear_model.LogisticRegression(C=100)
rbm1 = BernoulliRBM(n_components=100, learning_rate=0.06, n_iter=100, verbose=1, random_state=101)
rbm2 = BernoulliRBM(n_components=90, learning_rate=0.06, n_iter=3, verbose=1, random_state=101)
rbm3 = BernoulliRBM(n_components=60, learning_rate=0.06, n_iter=100, verbose=1, random_state=101)
#DBN3 = Pipeline(steps=[('rbm1', rbm1),('rbm2', rbm2), ('rbm3', rbm3), ('logistic', logistic)])
DBN3 = Pipeline(steps=[('rbm1', rbm1),('rbm2', rbm2), ('rbm3', rbm3), ('logistic', logistic)])

DBN3.fit(encoded_train, Y_train)

print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(
        Y_test,
        DBN3.predict(encoded_test))))
t2= time.time()

print("time consumption is : ", t2-t1)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -20.29, time = 0.16s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -20.29, time = 0.27s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -20.29, time = 0.26s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -20.29, time = 0.26s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -20.29, time = 0.28s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -20.29, time = 0.28s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -20.29, time = 0.28s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -20.29, time = 0.27s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -20.29, time = 0.30s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -20.29, time = 0.30s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -20.29, time = 0.27s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -20.29, time = 0.30s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -20.29, time = 0.29s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -20.29, time = 0.26s
[BernoulliRBM] Iteration 15, 

[BernoulliRBM] Iteration 16, pseudo-likelihood = -0.07, time = 0.22s
[BernoulliRBM] Iteration 17, pseudo-likelihood = -0.07, time = 0.23s
[BernoulliRBM] Iteration 18, pseudo-likelihood = -0.09, time = 0.23s
[BernoulliRBM] Iteration 19, pseudo-likelihood = -0.07, time = 0.23s
[BernoulliRBM] Iteration 20, pseudo-likelihood = -0.08, time = 0.24s
[BernoulliRBM] Iteration 21, pseudo-likelihood = -0.08, time = 0.23s
[BernoulliRBM] Iteration 22, pseudo-likelihood = -0.08, time = 0.23s
[BernoulliRBM] Iteration 23, pseudo-likelihood = -0.07, time = 0.23s
[BernoulliRBM] Iteration 24, pseudo-likelihood = -0.08, time = 0.24s
[BernoulliRBM] Iteration 25, pseudo-likelihood = -0.08, time = 0.23s
[BernoulliRBM] Iteration 26, pseudo-likelihood = -0.06, time = 0.21s
[BernoulliRBM] Iteration 27, pseudo-likelihood = -0.06, time = 0.27s
[BernoulliRBM] Iteration 28, pseudo-likelihood = -0.06, time = 0.23s
[BernoulliRBM] Iteration 29, pseudo-likelihood = -0.06, time = 0.23s
[BernoulliRBM] Iteration 30, pseud

/home/alandoli/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/alandoli/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
